In [ ]:
from deepeval.synthesizer import Synthesizer
from custom_models import CustomLLModel, CustomEmbeddingModel
from synthetic_dataset_generate import DeepEvalSynthesizer
from utils.llm_con import get_chat_openai
from deepeval.dataset import EvaluationDataset as deepeval_EvaluationDataset
from langchain_core.messages import HumanMessage
from deepeval.test_case import LLMTestCase

from deepeval.metrics import HallucinationMetric

from deepeval import evaluate as deepeval_evaluate
llm=get_chat_openai()
deep_e=DeepEvalSynthesizer(CustomLLModel(llm),CustomEmbeddingModel())

In [ ]:
path='../../Datasets/txt_example.txt'

result=deep_e.generate_goldens_from_documents(document_paths=path)

In [ ]:
deep_e.to_dataframe()
deep_e.save_dataset(directory="../../Datasets/")


In [ ]:


class MeetingSummarizer:
    def __init__(
            self,

    ):
        self.client = llm
        self.prompt_template = (
            "You are an AI assistant tasked with summarizing legal documents"
            " concisely and accurately. Given the following legal text, generate"
            " a summary that captures the key points while avoiding unnecessary"
            " details. Ensure neutrality and refrain from interpreting beyond the"
            " provided text.\n\n{text}"
        )

    def summarize(self, transcript: str) -> str:
        """
        Summarize the provided transcript using the ChatOpenAI model.

        Args:
            transcript (str): The legal document text to summarize.

        Returns:
            str: The generated summary.
        """
        prompt = self.prompt_template.format(text=transcript.strip())

        response = self.client(
            [HumanMessage(content=prompt)]
        )

        return response.content.strip()
llm_=MeetingSummarizer()

In [ ]:

for golden in dataset.goldens:
    actual_output = llm_.summarize(golden.expected_output)
    dataset.add_test_case(
        LLMTestCase(
            input=golden.input,
            actual_output=actual_output,
            expected_output=golden.expected_output,
            context=golden.context

        )
    )

In [ ]:

metric =HallucinationMetric(threshold=0.5,model=CustomLLModel(llm))

deepeval_evaluate(test_cases=dataset, metrics=[metric])